In [1]:
import numpy as np
import pandas as pd

import wittgenstein as lw

from architecture.classifier import DeepBinaryClassifier
from architecture.nodes.ripper import make_ripper_node

In [2]:
train_df = pd.read_csv("./data/mnist/mnist_train.csv")
test_df = pd.read_csv("./data/mnist/mnist_test.csv")
column_names = train_df.drop(columns="label").columns

input_values_train = train_df.drop(columns="label").to_numpy(bool)
target_values_train = train_df["label"].to_numpy(bool)

input_values_test = test_df.drop(columns="label").to_numpy(bool)
target_values_test = test_df["label"].to_numpy(bool)


print("Dataset shapes:")
print(f"TRAIN | input: {input_values_train.shape}, target: {target_values_train.shape}, pos. frac.: {target_values_train.mean():.3f}")
print(f"TEST  | input: {input_values_test.shape}, target: {target_values_test.shape}, pos. frac.: {target_values_test.mean():.3f}")

Dataset shapes:
TRAIN | input: (10000, 196), target: (10000,), pos. frac.: 0.486
TEST  | input: (1500, 196), target: (1500,), pos. frac.: 0.497


In [3]:
input_values_train_df = pd.DataFrame(input_values_train, columns=column_names)
input_values_test_df  = pd.DataFrame(input_values_test, columns=column_names)
target_values_train_df = pd.Series(target_values_train, name="class")
target_values_test_df  = pd.Series(target_values_test, name="class")

ripper = lw.RIPPER(random_state=42)
%time ripper.fit(pd.concat([input_values_train_df, target_values_train_df], axis=1), class_feat="class", pos_class=True)

train_acc = ripper.score(input_values_train_df, target_values_train_df)
test_acc  = ripper.score(input_values_test_df, target_values_test_df)

print(f"Test accuracy:     {test_acc:.3f}")

CPU times: user 41.8 s, sys: 449 ms, total: 42.2 s
Wall time: 42.4 s
Test accuracy:     0.887


In [4]:
layer_node_counts = [32] * 4
config = dict(layer_node_counts=layer_node_counts, layer_bit_counts=[16]*len(layer_node_counts), seed=42)

net = DeepBinaryClassifier(**config, node_factory=make_ripper_node, jobs=8)
%time net.fit(input_values_train, target_values_train)

pred_values_test = net.predict(input_values_test)
ref_values_test = np.broadcast_to(target_values_test[:, None], pred_values_test.shape)
accuracies_test = (pred_values_test == ref_values_test).mean(axis=0)

best_output_node_idx = int(np.argmax(accuracies_test))
best_output_node_name = net.node_names[-1][best_output_node_idx]

print("Best node:", best_output_node_name, "with accuracy", accuracies_test[best_output_node_idx])

CPU times: user 930 ms, sys: 247 ms, total: 1.18 s
Wall time: 3min 47s
Best node: L4N11 with accuracy 0.87
